In [18]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from PIL import Image
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import fits

import numpy.ma as ma
from astropy.wcs import WCS


import sys 

sys.path.insert(1, 'C:/Users/boibr/OneDrive/Documents/GitHub/MaNGA-VelMap/')
from GenerateCutout import get_cutout

In [19]:
data_folder = "/Users/Emlilio/School/Internships/UR ASTRO/Data/" #create a variable for directory to common folder

In [20]:
drpall_fn = data_folder + "drpall_ttype_R90.fits"
drpall = Table.read(drpall_fn, format="fits",hdu=1)

In [31]:
drpall_dict = {}                    #create the dictionary

for i in range(len(drpall)):           #loop that repeats for the length of drpalltt file
    plateifu = drpall['plateifu'][i]    #variable that holds the plateifu withing drpalltt at i(current row) 
    drpall_dict[plateifu] = i     

In [45]:
max_lim = 400              #creates default plot limit

for i in range(9080,9280):
    plateifu = drpall['plateifu'][i]      #calls the specific row by index
    try:
        cube_df = data_folder + 'MaNGA/manga-'+ plateifu +'-MAPS-HYB10-MILESHC-MASTARSSP.fits.gz'
        cube = fits.open(cube_df)                        #file requires opening
        stellar_vel = cube['STELLAR_VEL'].data           #for the following we create variable representing data from file HDU's
        stellar_vel_ivar = cube['STELLAR_VEL_IVAR'].data 
        stellar_mask = cube['STELLAR_VEL_MASK'].data
        halpha_vel = cube['EMLINE_GVEL'].data[23]        #the following three have several channels, but were only concerned with the Halpha spectra (channel 23)
        halpha_gvel_ivar = cube['EMLINE_GVEL_IVAR'].data[23]
        halpha_gvel_mask = cube['EMLINE_GVEL_MASK'].data[23]
        flux = cube['SPX_MFLUX'].data
    
        cube.close()
    #creates figure for subplots,  
        fig = plt.figure( figsize=(15,15))
        fig.suptitle(plateifu)
        ra = drpall['objra'][i]
        dec = drpall['objdec'][i]
        r90 = drpall['R90'][i]
        directory = data_folder + 'Plots/Cutouts/'
        gal_image, wcs = get_cutout(plateifu,ra,dec,r90,directory) 
        img = np.asarray(Image.open(data_folder + 'Plots/Cutouts/'+plateifu+'.jpg'))
    
    
        index = drpall_dict[plateifu]
    
        center_coords = SkyCoord(ra*u.deg,dec*u.deg)
    #center_coords
    
        pos_angle = drpall['nsa_elpetro_phi'][index]          #position angle is in degrees, this converts to radians bc math expects radians
        
        
        axis_ratio = drpall['nsa_sersic_ba'][index]
    
        point1 = center_coords.directional_offset_by(pos_angle*u.deg,r90*u.arcsec)
    
        alpha1 = point1.ra.deg
        delta1 = point1.dec.deg
        
        
        point2 = center_coords.directional_offset_by((pos_angle+180)*u.deg,r90*u.arcsec)
        
        alpha2 = point2.ra.deg
        delta2 = point2.dec.deg
    
        point3 = center_coords.directional_offset_by((pos_angle+90)*u.deg,(r90*axis_ratio)*u.arcsec)
        
        alpha3 = point3.ra.deg
        delta3 = point3.dec.deg
        
        
        point4 = center_coords.directional_offset_by((pos_angle-90)*u.deg,(r90*axis_ratio)*u.arcsec)
        
        alpha4 = point4.ra.deg
        delta4 = point4.dec.deg
    
    
        mhalpha_vel = ma.array(halpha_vel, mask = stellar_mask)
        
      #  set equal limits for color bar
        val_max = mhalpha_vel.max()
        val_min = mhalpha_vel.min()
        if (val_max >= abs(val_min)):
            lim = val_max
        else:
            lim = abs(val_min)
        
        if (lim > max_lim):
            lim = max_lim
    
        #create subplot for mhalpha_vel
        ax2 = fig.add_subplot(222)
        im = ax2.imshow(mhalpha_vel, cmap ='bwr', origin = 'lower',vmax = lim, vmin = -lim)
        ax2.set_xlabel('spaxel')
        ax2.set_ylabel('spaxel')
        ax2.set_title('Masked H-alpha Velocity')
        
        
        fig.colorbar(im, ax = ax2,label = 'Velocity [km/s]' )
        #plt.close()
    
    
        mstellar_vel = ma.array(stellar_vel, mask = stellar_mask)
    
        #create subplot for mhalpha_vel
        val_max = mstellar_vel.max()
        val_min = mstellar_vel.min()
        if (val_max >= abs(val_min)):
            lim = val_max
        else:
            lim = abs(val_min)
    
        if (lim > max_lim):
            lim = max_lim
        #create subplot for mstellar_vel
        ax1 = fig.add_subplot(221)
        im = ax1.imshow(mstellar_vel, cmap ='bwr', origin = 'lower', vmax = lim, vmin = -lim )
        ax1.set_xlabel('spaxel')
        ax1.set_ylabel('spaxel')
        ax1.set_title('Masked Stellar Velocity')
        
        fig.colorbar(im, ax = ax1,label = 'Velocity [km/s]')
    
        #create subplot for flux
    
        ax4 = fig.add_subplot(224)
        im = ax4.imshow(flux, cmap ='hot', origin = 'lower', vmin = 0)
        ax4.set_title('Flux')
        ax4.set_xlabel('spaxel')
        ax4.set_ylabel('spaxel')
        
        
        
        fig.colorbar(im, ax = ax4,label = '10-17 erg/s/cm2/Å/spaxel' )
    
        ax3 = fig.add_subplot(223, projection=wcs)
    
    
        im = ax3.imshow(img[::-1])
        ax3.grid(color='white', ls='solid')
        ax3.set(xlabel='Galactic Longitude', ylabel='Galactic Latitude')
        ax3.plot([alpha1,alpha2],[delta1,delta2],'g',transform = ax3.get_transform('world'))
        ax3.plot([alpha3,alpha4],[delta3,delta4],'g',transform = ax3.get_transform('world'))
        ax3.plot(center_coords.ra.deg,center_coords.dec.deg,'rx',transform = ax3.get_transform('world') )
       
        plt.savefig(data_folder + 'Plots/4x4_plots/'+ plateifu +'_4x4.png')
        plt.close(fig)
           
            
    except:
        continue

<Response [500]>


<Figure size 1500x1500 with 0 Axes>